为了运行以下笔记本，如果你还没有这样做，需要在.env文件中将openai密钥设置为`OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

接下来，我们将把嵌入索引加载到一个 Pandas Dataframe 中。嵌入索引存储在一个名为 `embedding_index_3m.json` 的 JSON 文件中。嵌入索引包含截至 2023 年 10 月底的每个 YouTube 转录的嵌入。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

接下来，我们将创建一个名为 `get_videos` 的函数，用于在嵌入索引中搜索查询内容。该函数会返回与查询最相似的前 5 个视频。函数的工作流程如下：

1. 首先，创建嵌入索引的副本。
2. 接着，使用 OpenAI Embedding API 计算查询的嵌入向量。
3. 然后，在嵌入索引中新增一列，名为 `similarity`。`similarity` 列存储查询嵌入与每个视频片段嵌入之间的余弦相似度。
4. 接下来，根据 `similarity` 列筛选嵌入索引。只保留余弦相似度大于等于 0.75 的视频。
5. 最后，按照 `similarity` 列进行排序，返回最相似的前 5 个视频。


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

这个函数非常简单，它只是打印出搜索查询的结果。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. 首先，将嵌入索引加载到一个 Pandas Dataframe 中。
2. 接下来，系统会提示用户输入查询内容。
3. 然后会调用 `get_videos` 函数，在嵌入索引中搜索该查询。
4. 最后会调用 `display_results` 函数，将结果展示给用户。
5. 随后会再次提示用户输入新的查询。这个流程会一直持续，直到用户输入 `exit`。

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.zh.png)

系统会提示你输入查询内容。输入你的问题并按回车。应用程序会返回一组与查询相关的视频，还会提供一个链接，直接跳转到视频中回答该问题的片段。

你可以尝试以下查询：

- 什么是 Azure 机器学习？
- 卷积神经网络是如何工作的？
- 什么是神经网络？
- 我可以在 Azure 机器学习中使用 Jupyter Notebooks 吗？
- 什么是 ONNX？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。我们力求准确，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文件应被视为权威来源。对于关键信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或曲解，我们概不负责。
